# 다중공선성 확인을 위한 회귀분석

종속변수 (Y) : 월별 자동차 판매량

독립변수(X):


그룹 1 소비자 심리):
차량 구매에 대한 소비자의 심리를 나타내는 지표
- 월별 Google Trend (RAV4 월별 Google 검색량)
- 월별 소비자 신뢰 지수 (University of Michigan Consumer Sentiment Index )
: 소비자들이 경제 상황에 대해 느끼는 감정. 낙관적인지 비관적인지. 숫자가 높을 수록 시장 성장에 대해 낙관적인 감정.

그룹 2: 리뷰 데이터
- Edmund, Car.com에서 수집한 Toyota RAV4 소비자 리뷰
- 연 단위  JD Power (RAV4에 대한 10점 척도 점수, 실 사용자가 매긴 평점)

그룹 3: 거시 경제 지표 :
자동차 구매 시 소비자의 실제 경제 상황을 나타내는 지표들
- 분기별 Median usual weekly nominal earnings
- 월별 실업률 Unemployment Rate
- 월별 휘발유 가격
- 월별 금리

그룹 3: 시장 변수 :
: 자동차 시장에 대한 지표 (판매량, 소비자 수의 증감)  
- 월별 인구 수 (미국의 인구 증가에 성인 이상의 이민자가 기여하는 비율이 높음)
- 월별 전체 자동차 판매량
- 경쟁 모델 월별 판매량 (Honda CR-V)
- 경쟁 모델 월별 판매량 (Chevrolet Equinox)     

그룹 4: 기타  
- 연  단위 RAV4 차량 가격






: Ravo4 (미국시장)의 경우 고객이 해당 차량을 주문한 이후 인도받는데까지 시간이 소요되지 않고, 당일에 바로 가능함   
-> 종속 변수와 독립변수간의 시간 차이를 두지 않음

## 분석 과정

1. 월별 자동차 판매량 데이터 불러오기
2. 감성분석 결과 -> 월별 감성점수 계산
  - 감성점수(sentiment_prediction)을 독립변수로 추가
3. 독립변수 + 종속변수 데이터 병합

4. 회귀분석 -> 다중공선성 검토
5. 변수 선택 및 새로운 변수 생성
6. 최종 변수로 다중공선성 재검토

In [75]:
import pandas as pd

In [77]:
# 고객 리뷰 감성 점수 불러오기
reviews = pd.read_excel('data/sentiment_1.xlsx')

In [79]:
data = pd.read_csv('data/RAV4_data.csv')

/var/folders/3g/tlt8qmcx1nj7kqtx38v8dw380000gn/T/ipykernel_19819/801452899.py:1: DtypeWarning: Columns (0,1,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/RAV4_data.csv')


In [81]:
data.head()

,Date,RAV4 Sales,Google Trend,JD Power,Population in Milion,Consumer Sentiment,Federal Funds Effective Rate,Median usual weekly real earnings,Unemployment Rate,Gas Price,Competitor Sales (Honda CR-V),Competitor Sales (Chevrolet Equinox),Total Vehicle Sales (In milions),Price (MSRP)
0,Jan-10,"7,894",23.0,84,308706.0,74.4,0.11,344.0,9.8,2.71500,"9,672","9,513",10.893,"24,943"
1,Feb-10,"6,799",20.0,84,308904.0,73.6,0.13,344.0,9.8,2.64400,"11,828","8,061",10.315,"24,943"
2,Mar-10,"25,781",24.0,84,309095.0,73.6,0.16,344.0,9.9,2.77160,"14,848","12,805",11.772,"24,943"
3,Apr-10,"14,851",20.0,84,309289.0,72.2,0.20,342.0,9.9,2.84825,"16,661","11,987",11.454,"24,943"
4,May-10,"13,102",21.0,84,309485.0,73.6,0.20,342.0,9.6,2.83620,"17,820","13,134",12.030,"24,943"


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032360 entries, 0 to 1032359
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Date                                  168 non-null    object 
 1   RAV4 Sales                            168 non-null    object 
 2   Google Trend                          168 non-null    float64
 3   JD Power                              10815 non-null  object 
 4   Population in Milion                  168 non-null    float64
 5   Consumer Sentiment                    168 non-null    float64
 6   Federal Funds Effective Rate          168 non-null    float64
 7   Median usual weekly real earnings     168 non-null    float64
 8   Unemployment Rate                     168 non-null    float64
 9   Gas Price                             168 non-null    float64
 10  Competitor Sales (Honda CR-V)         168 non-null    object 
 11  Competitor 

In [85]:
#Date를 year_month로 바꾸기
data['year_month'] = pd.to_datetime(data['Date'], format="%b-%y")


#Date 행 삭제
data = data.drop(columns=['Date'])

#year_month에 day는 제외시키기
data['year_month'] = data['year_month'].dt.to_period('M')

In [87]:
cols = ['year_month'] + [col for col in data.columns if col != 'year_month']
data = data[cols]

In [89]:
#data 2023년 12월 이후 데이터 삭제
data = data[data['year_month'] <'2024-01']


In [91]:
data.head(2)

,year_month,RAV4 Sales,Google Trend,JD Power,Population in Milion,Consumer Sentiment,Federal Funds Effective Rate,Median usual weekly real earnings,Unemployment Rate,Gas Price,Competitor Sales (Honda CR-V),Competitor Sales (Chevrolet Equinox),Total Vehicle Sales (In milions),Price (MSRP)
0,2010-01,"7,894",23.0,84,308706.0,74.4,0.11,344.0,9.8,2.715,"9,672","9,513",10.893,"24,943"
1,2010-02,"6,799",20.0,84,308904.0,73.6,0.13,344.0,9.8,2.644,"11,828","8,061",10.315,"24,943"


In [93]:
#월별 감성 점수 계산
reviews['date'] = pd.to_datetime(reviews['date'])

In [95]:
# 년-월로 그룹화하여 긍정 리뷰 비율 계산
reviews['year_month'] = reviews['date'].dt.to_period('M')
sentiment = reviews.groupby('year_month')['sentiment_prediction'].mean().reset_index()

# 월별 감성 점수를 dataframe으로 변환
sentiment['year_month'] = sentiment['year_month'].astype(str)

#monthly_sentiment 2024년의 행 제거
sentiment = sentiment[sentiment['year_month'] <'2024-01']

In [97]:
sentiment.head()

,year_month,sentiment_prediction
0,2010-01,1.0
1,2010-02,1.0
2,2010-03,1.0
3,2010-04,1.0
4,2010-05,1.0


In [99]:
# Creating the full date range from 2010-01 to 2023-12
full_date_range = pd.date_range(start='2010-01-01', end='2023-12-01', freq='MS').strftime('%Y-%m')

# Create a DataFrame with the full date range
df_full_date_range = pd.DataFrame(full_date_range, columns=['year_month'])

In [101]:
# Merge to find missing dates and fill with NaN where necessary
df_complete_sentiment = pd.merge(df_full_date_range, sentiment, on='year_month', how='left')

# Ensure the year_month columns in both DataFrames are of the same type (string)
data['year_month'] = data['year_month'].astype(str)
df_complete_sentiment['year_month'] = df_complete_sentiment['year_month'].astype(str)

# Merge the data DataFrame with the complete sentiment DataFrame
merged_data = pd.merge(data, df_complete_sentiment, on='year_month', how='left')

In [103]:
merged_data[:6]

,year_month,RAV4 Sales,Google Trend,JD Power,Population in Milion,Consumer Sentiment,Federal Funds Effective Rate,Median usual weekly real earnings,Unemployment Rate,Gas Price,Competitor Sales (Honda CR-V),Competitor Sales (Chevrolet Equinox),Total Vehicle Sales (In milions),Price (MSRP),sentiment_prediction
0,2010-01,"7,894",23.0,84,308706.0,74.4,0.11,344.0,9.8,2.71500,"9,672","9,513",10.893,"24,943",1.0
1,2010-02,"6,799",20.0,84,308904.0,73.6,0.13,344.0,9.8,2.64400,"11,828","8,061",10.315,"24,943",1.0
2,2010-03,"25,781",24.0,84,309095.0,73.6,0.16,344.0,9.9,2.77160,"14,848","12,805",11.772,"24,943",1.0
3,2010-04,"14,851",20.0,84,309289.0,72.2,0.20,342.0,9.9,2.84825,"16,661","11,987",11.454,"24,943",1.0
4,2010-05,"13,102",21.0,84,309485.0,73.6,0.20,342.0,9.6,2.83620,"17,820","13,134",12.030,"24,943",1.0
5,2010-06,"12,573",22.0,84,309691.0,76.0,0.18,342.0,9.4,2.73150,"16,041","11,490",11.598,"24,943",NaN


In [105]:
#sentiment_prediction에서 결측값을 0으로 채워넣기
merged_data['sentiment_prediction'] = merged_data['sentiment_prediction'].fillna(0)

In [107]:
# 쉼표 제거 및 데이터 타입 변환
merged_data['RAV4 Sales'] = merged_data['RAV4 Sales'].str.replace(',', '').astype(float)
merged_data['Competitor Sales (Honda CR-V)'] = merged_data['Competitor Sales (Honda CR-V)'].str.replace(',', '').astype(float)
merged_data['Competitor Sales (Chevrolet Equinox)'] = merged_data['Competitor Sales (Chevrolet Equinox)'].str.replace(',', '').astype(float)
merged_data['Price (MSRP)'] = merged_data['Price (MSRP)'].str.replace(',', '').astype(float)

In [109]:
#데이터별 타입 확인
merged_data.dtypes

year_month                               object
RAV4 Sales                              float64
Google Trend                            float64
JD Power                                 object
Population in Milion                    float64
Consumer Sentiment                      float64
Federal Funds Effective Rate            float64
Median usual weekly real earnings       float64
Unemployment Rate                       float64
Gas Price                               float64
Competitor Sales (Honda CR-V)           float64
Competitor Sales (Chevrolet Equinox)    float64
Total Vehicle Sales (In milions)        float64
Price (MSRP)                            float64
sentiment_prediction                    float64
dtype: object

In [159]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   year_month                            168 non-null    object 
 1   RAV4 Sales                            168 non-null    float64
 2   Google Trend                          168 non-null    float64
 3   JD Power                              168 non-null    object 
 4   Population in Milion                  168 non-null    float64
 5   Consumer Sentiment                    168 non-null    float64
 6   Federal Funds Effective Rate          168 non-null    float64
 7   Median usual weekly real earnings     168 non-null    float64
 8   Unemployment Rate                     168 non-null    float64
 9   Gas Price                             168 non-null    float64
 10  Competitor Sales (Honda CR-V)         168 non-null    float64
 11  Competitor Sales (C

## Check for Multicollinearity / 다중공선성 확인

In [112]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [114]:
# 독립변수 선택 (종속변수 'RAV4 Sales'는 제외)
X = merged_data[['Google Trend', 'JD Power', 'Population in Milion', 'Consumer Sentiment', 'Federal Funds Effective Rate',
        'Median usual weekly real earnings', 'Unemployment Rate', 'Gas Price', 'Competitor Sales (Honda CR-V)',
        'Competitor Sales (Chevrolet Equinox)', 'Total Vehicle Sales (In milions)', 'Price (MSRP)', 'sentiment_prediction']]


In [116]:
#데이터들을 숫자형으로 변환
X = X.apply(pd.to_numeric, errors='coerce')

In [118]:
#X에 결측치가 있는지 확인
X.isnull().sum()

Google Trend                            0
JD Power                                0
Population in Milion                    0
Consumer Sentiment                      0
Federal Funds Effective Rate            0
Median usual weekly real earnings       0
Unemployment Rate                       0
Gas Price                               0
Competitor Sales (Honda CR-V)           0
Competitor Sales (Chevrolet Equinox)    0
Total Vehicle Sales (In milions)        0
Price (MSRP)                            0
sentiment_prediction                    0
dtype: int64

In [120]:
# VIF 계산
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

In [122]:
from tabulate import tabulate
print(tabulate(vif_data, headers='keys', tablefmt='psql'))

+----+--------------------------------------+-------------+
|    | feature                              |         VIF |
|----+--------------------------------------+-------------|
|  0 | Google Trend                         |    91.0564  |
|  1 | JD Power                             |  2231.18    |
|  2 | Population in Milion                 | 24167.1     |
|  3 | Consumer Sentiment                   |   223.564   |
|  4 | Federal Funds Effective Rate         |     3.39046 |
|  5 | Median usual weekly real earnings    | 20847.9     |
|  6 | Unemployment Rate                    |    91.7945  |
|  7 | Gas Price                            |   114.292   |
|  8 | Competitor Sales (Honda CR-V)        |    42.8972  |
|  9 | Competitor Sales (Chevrolet Equinox) |    32.2918  |
| 10 | Total Vehicle Sales (In milions)     |   457.439   |
| 11 | Price (MSRP)                         |  2058.11    |
| 12 | sentiment_prediction                 |    43.1688  |
+----+----------------------------------

### 다중공선성 VIF 해석 기준


*   **VIF < 5**: 다중공선성이 거의 없는 것으로 간주
*   **5 <= VIF < 10**: 다중공선성이 있을 수 있지만 심각하지는 않음
*   **VIF >= 10:** 다중공선성이 높음

In [125]:
# VIF report
high_multicollinearity = vif_data[vif_data['VIF'] >= 10]
moderate_multicollinearity = vif_data[(vif_data['VIF'] >= 5) & (vif_data['VIF'] < 10)]
low_multicollinearity = vif_data[vif_data['VIF'] < 5]

print("High Multicollinearity (VIF >= 10):")
print(tabulate(high_multicollinearity, headers='keys', tablefmt='psql'))
print("\nModerate Multicollinearity (5 <= VIF < 10):")
print(tabulate(moderate_multicollinearity, headers='keys', tablefmt='psql'))
print("\nLow Multicollinearity (VIF < 5):")
print(tabulate(low_multicollinearity, headers='keys', tablefmt='psql'))


High Multicollinearity (VIF >= 10):
+----+--------------------------------------+------------+
|    | feature                              |        VIF |
|----+--------------------------------------+------------|
|  0 | Google Trend                         |    91.0564 |
|  1 | JD Power                             |  2231.18   |
|  2 | Population in Milion                 | 24167.1    |
|  3 | Consumer Sentiment                   |   223.564  |
|  5 | Median usual weekly real earnings    | 20847.9    |
|  6 | Unemployment Rate                    |    91.7945 |
|  7 | Gas Price                            |   114.292  |
|  8 | Competitor Sales (Honda CR-V)        |    42.8972 |
|  9 | Competitor Sales (Chevrolet Equinox) |    32.2918 |
| 10 | Total Vehicle Sales (In milions)     |   457.439  |
| 11 | Price (MSRP)                         |  2058.11   |
| 12 | sentiment_prediction                 |    43.1688 |
+----+--------------------------------------+------------+

Moderate Multicolli

## 다중공선성 해결을 위한 변수 제거


In [128]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [130]:
#X 에서 populPopulation in Milion 제거
X = X.drop(columns=['Population in Milion'])

#X 에서 'Median usual weekly real earnings' 제거
X = X.drop(columns=['Median usual weekly real earnings'])


In [132]:
# VIF 계산
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

In [134]:
from tabulate import tabulate
print(tabulate(vif_data, headers='keys', tablefmt='psql'))

+----+--------------------------------------+------------+
|    | feature                              |        VIF |
|----+--------------------------------------+------------|
|  0 | Google Trend                         |   53.4702  |
|  1 | JD Power                             | 1131.17    |
|  2 | Consumer Sentiment                   |  191.847   |
|  3 | Federal Funds Effective Rate         |    3.19848 |
|  4 | Unemployment Rate                    |   28.2879  |
|  5 | Gas Price                            |   49.0574  |
|  6 | Competitor Sales (Honda CR-V)        |   40.3078  |
|  7 | Competitor Sales (Chevrolet Equinox) |   32.0084  |
|  8 | Total Vehicle Sales (In milions)     |  275.315   |
|  9 | Price (MSRP)                         |  899.919   |
| 10 | sentiment_prediction                 |   42.6366  |
+----+--------------------------------------+------------+


In [136]:
X = X.drop(columns=['JD Power'])

In [138]:
#Goole Trend 와 Consumer Sentiment를 PCA로 합쳐서 새로운 변수 생성


variables_for_pca = ['Google Trend', 'Consumer Sentiment']

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X[variables_for_pca])

pca = PCA(n_components=1)
principal_component = pca.fit_transform(scaled_data)

# Step 3: Add the new PCA variable back to X
X['Consumer_Interest'] = principal_component

# Step 4: Calculate VIF for each variable in the new dataframe
# Dropping the original variables used in PCA to avoid multicollinearity in VIF calculation
X_vif = X.drop(columns=variables_for_pca)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Display VIF
print(vif_data)

                               Variable         VIF
0          Federal Funds Effective Rate    3.045608
1                     Unemployment Rate   14.658000
2                             Gas Price   26.540335
3         Competitor Sales (Honda CR-V)   37.624838
4  Competitor Sales (Chevrolet Equinox)   28.507704
5      Total Vehicle Sales (In milions)  230.748905
6                          Price (MSRP)  204.549997
7                  sentiment_prediction   42.069757
8                     Consumer_Interest    4.122419


In [140]:

# PCA on 'Competitor Sales (Honda CR-V)', 'Competitor Sales (Chevrolet Equinox)', 'Total Vehicle Sales (In milions)'
additional_variables_for_pca = ['Competitor Sales (Honda CR-V)', 'Competitor Sales (Chevrolet Equinox)', 'Total Vehicle Sales (In milions)']

# Scaling the data for additional PCA
additional_scaled_data = scaler.fit_transform(X[additional_variables_for_pca])

# Performing PCA
additional_pca = PCA(n_components=1)
additional_principal_component = additional_pca.fit_transform(additional_scaled_data)

# Adding the new PCA variable back to X
X['Competitor_Sales'] = additional_principal_component

# Step 6: Perform PCA on 'Gas Price' and 'Price (MSRP)'
gas_price_and_msrp = ['Gas Price', 'Price (MSRP)']

# Scaling the data for PCA
gas_msrp_scaled_data = scaler.fit_transform(X[gas_price_and_msrp])

# Performing PCA
gas_msrp_pca = PCA(n_components=1)
gas_msrp_principal_component = gas_msrp_pca.fit_transform(gas_msrp_scaled_data)

# Adding the new PCA variable back to X with a suitable name
X['Price_Factor'] = gas_msrp_principal_component


# Step 8: Calculate VIF for each variable in the new dataframe
# Dropping the original variables used in PCA to avoid multicollinearity in VIF calculation
X_vif = X.drop(columns=variables_for_pca + additional_variables_for_pca + gas_price_and_msrp)


In [142]:
#Unemployment Rate 제거
X_vif = X_vif.drop(columns=['Unemployment Rate'])

In [144]:
# VIF 재검토
vif_data = pd.DataFrame()
vif_data['Variable'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]


print(vif_data)

                       Variable       VIF
0  Federal Funds Effective Rate  2.368529
1          sentiment_prediction  1.713421
2             Consumer_Interest  1.779507
3              Competitor_Sales  1.543694
4                  Price_Factor  1.329435


# 최종 변수 목록:


*   Federal Funds Effective Rate
*   sentiment_prediction

[새로운 변수]
- Consumer_Interest (Google Trend와 Consumer Sentiment의 PCA 결합)
- Competitor_Sales (Competitor Sales (Honda CR-V), Competitor Sales (Chevrolet Equinox), Total Vehicle Sales (In milions)의 PCA 결합)
- Price_Factor (Gas Price와 Price (MSRP)의 PCA 결합)

In [148]:
X_vif['year_month'] = merged_data['year_month']
X_vif['RAV4 Sales'] = merged_data['RAV4 Sales']

In [150]:
columns = ['year_month', 'RAV4 Sales'] + [col for col in X_vif if col not in ['year_month', 'RAV4 Sales']]
X_vif = X_vif[columns]

In [152]:
final_data = X_vif.copy()
final_data.head()

,year_month,RAV4 Sales,Federal Funds Effective Rate,sentiment_prediction,Consumer_Interest,Competitor_Sales,Price_Factor
0,2010-01,7894.0,0.11,1.0,-0.452485,3.869561,-0.592358
1,2010-02,6799.0,0.13,1.0,-0.495998,3.995724,-0.511696
2,2010-03,25781.0,0.16,1.0,-0.380068,2.875564,-0.656660
3,2010-04,14851.0,0.20,1.0,-0.419988,2.895012,-0.743742
4,2010-05,13102.0,0.20,1.0,-0.467016,2.524651,-0.730052


In [154]:
#엑셀 파일로 내보내기
final_data.to_excel('final_data.xlsx', index=False)



---

